![](2022-10-19-15-46-52.png)

![](2022-10-19-16-10-48.png)

In [1]:
import pandas as pd

from ortools.sat.python import cp_model

import numpy as np

In [2]:
## Input

nodes = pd.read_excel('data/route_inputs.xlsx',sheet_name = 'nodes')
paths = pd.read_excel('data/route_inputs.xlsx',sheet_name = 'paths')

In [3]:
nodes

,node,description
0,1,origin
1,2,middle point
2,3,middle point
3,4,middle point
4,5,middle point
5,6,middle point
6,7,destination


In [4]:
paths

,node_from,node_to,distance
0,1,2,220
1,1,3,1500
2,2,4,650
3,2,5,900
4,4,7,500
5,5,7,400
6,3,6,500
7,6,7,400


In [5]:
n_nodes = len(nodes)

print(f"Number of nodes: {n_nodes}")

n_paths = len(paths)

print(f"Number of paths: {n_paths}")

Number of nodes: 7
Number of paths: 8


In [6]:
# Model

model = cp_model.CpModel()

In [7]:
x  = np.zeros(n_paths).tolist()
x

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [8]:
for p in paths.index:
    x[p] = model.NewIntVar(0,1,f'x{p}')
    
x

[x0(0..1),
 x1(0..1),
 x2(0..1),
 x3(0..1),
 x4(0..1),
 x5(0..1),
 x6(0..1),
 x7(0..1)]

In [9]:
# Ojective function
objective_func = sum([x[p]*paths.distance[p] for p in paths.index])
model.Minimize(objective_func)
objective_func

Sum(Sum(Sum(Sum(Sum(Sum(Sum(ProductCst(x0(0..1), 220), ProductCst(x1(0..1), 1500)), ProductCst(x2(0..1), 650)), ProductCst(x3(0..1), 900)), ProductCst(x4(0..1), 500)), ProductCst(x5(0..1), 400)), ProductCst(x6(0..1), 500)), ProductCst(x7(0..1), 400))

In [10]:
# Constraints

node_origin = int(nodes.node[nodes.description=='origin'])
node_destination = int(nodes.node[nodes.description=='destination'])

In [11]:
print(node_origin)
print(node_destination)

1
7


In [12]:
model.Add(sum([x[p] for p in paths.index[paths.node_from==node_origin]])==1)
model.Add(sum([x[p] for p in paths.index[paths.node_to==node_destination]])==1)

In [13]:
for node in nodes.node[nodes.description=='middle point']:
    sum_in = sum([x[p] for p in paths.index[paths.node_to==node]])
    sum_out = sum([x[p] for p in paths.index[paths.node_from==node]])
    model.Add(sum_in == sum_out )

In [14]:
solver = cp_model.CpSolver()

status = solver.Solve(model)

In [15]:
print('status = ',solver.StatusName(status))
print('OF = ', solver.ObjectiveValue())

status =  OPTIMAL
OF =  1370.0


In [16]:
paths['activated'] = 0

for p in paths.index:
    paths.activated[p] = solver.Value(x[p])

paths

,node_from,node_to,distance,activated
0,1,2,220,1
1,1,3,1500,0
2,2,4,650,1
3,2,5,900,0
4,4,7,500,1
5,5,7,400,0
6,3,6,500,0
7,6,7,400,0


### Solving with pyomo

In [77]:
import pandas as pd
import numpy as np

import pyomo.environ as pyo
from pyomo.opt import SolverFactory

In [50]:
## Input

nodes = pd.read_excel('data/route_inputs.xlsx',sheet_name = 'nodes')
paths = pd.read_excel('data/route_inputs.xlsx',sheet_name = 'paths')

In [51]:
nodes

,node,description
0,1,origin
1,2,middle point
2,3,middle point
3,4,middle point
4,5,middle point
5,6,middle point
6,7,destination


In [52]:
paths

,node_from,node_to,distance
0,1,2,220
1,1,3,1500
2,2,4,650
3,2,5,900
4,4,7,500
5,5,7,400
6,3,6,500
7,6,7,400


In [72]:
## Creating variables

n_nodes = len(nodes)
n_paths = len(paths)

In [73]:
model = pyo.ConcreteModel()

In [74]:
model.x = pyo.Var(range(0, n_paths+1), bounds = (0,1), within = pyo.Binary)
x = model.x

In [75]:
## Objective function

model.obj = pyo.Objective(expr = sum([x[p]*paths.distance[p] for p in paths.index]), sense = pyo.minimize)

In [76]:
## Constraints

node_origin = int(nodes.node[nodes.description=='origin'])
node_destination = int(nodes.node[nodes.description=='destination'])

model.C1 = pyo.Constraint(expr = sum([x[p] for p in paths.index[paths.node_from==node_origin]])==1)
model.C2 = pyo.Constraint(expr = sum([x[p] for p in paths.index[paths.node_to==node_destination]])==1)

model.C3 = pyo.ConstraintList()

middle_nodes = nodes.node[nodes.description=='middle point']

for node in middle_nodes:
    sum_in = sum([x[p] for p in paths.index[paths.node_to==node]])
    sum_out = sum([x[p] for p in paths.index[paths.node_from==node]])

    model.C3.add(expr = sum_in==sum_out)

In [78]:
opt = SolverFactory('gurobi')

results = opt.solve(model)

In [79]:
model.pprint()

2 Set Declarations
    C3_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    x_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    9 : {0, 1, 2, 3, 4, 5, 6, 7, 8}

1 Var Declarations
    x : Size=9, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :   1.0 :     1 : False : False : Binary
          1 :     0 :   0.0 :     1 : False : False : Binary
          2 :     0 :   1.0 :     1 : False : False : Binary
          3 :     0 :   0.0 :     1 : False : False : Binary
          4 :     0 :   1.0 :     1 : False : False : Binary
          5 :     0 :   0.0 :     1 : False : False : Binary
          6 :     0 :   0.0 :     1 : False : False : Binary
          7 :     0 :   0.0 :     1 : False : False : Binary
          8 :     0 :  None :     1 : False :  True : Binary


In [80]:
for p in paths.index:
    print(f"p --> {pyo.value(x[p])}")

p --> 1.0
p --> 0.0
p --> 1.0
p --> 0.0
p --> 1.0
p --> 0.0
p --> 0.0
p --> 0.0
